In [55]:
from matching.games import HospitalResident
import random;

def del_player(p, choices, hospital_preferences):
    del choices[p]
    for (h, pl) in hospital_preferences.items():
        if p in pl:
            pl.remove(p)

def del_hospital(h, choices, hospital_preferences, capacities):
    del hospital_preferences[h]
    del capacities[h]
    for (p, hl) in choices.items():
        if h in hl:
            hl.remove(h)
def cleanup(choices, hospital_preferences, capacities):
    #cleanup : delete hospital and players that are finished
        cleanup = True
        while cleanup: 
            cleanup = False
            to_remove = []
            for (h, p) in hospital_preferences.items():
                #if an hospital is full or noone wants to go there, delete it
                if not (capacities[h] > 0 and len(p) > 0):
                    to_remove.append(h)
                    cleanup = True
            for h in to_remove:
                del_hospital(h, choices, hospital_preferences, capacities)
            
            #if a player don't want anything anymore, delete it
            to_remove = []
            for (p, hl) in choices.items():
                if len(hl) == 0:
                    to_remove.append(p)
                    cleanup = True    
            for p in to_remove:
                del_player(p, choices, hospital_preferences)  

def remove_conflict(choices, conflits, activite):
    for a in conflits[str(activite)]:
        if a in choices:
            choices.remove(a)

def match(choices, capacities, conflits):
    players = []
    hospital_preferences = dict()
    assignments = dict()
    #fill the assignments and hospital pref with empty lists
    for (h, c) in capacities.items():
        assignments[h] = []
        hospital_preferences[h] = []
    #add all players that wanted an hospital to that hospital's preference list
    for (p, c) in choices.items():
        for h in c:
            hospital_preferences[h].append(p)
    #shuffle the hospital preference list
    for (h, p) in hospital_preferences.items():
            random.shuffle(p)
            c = 0
            for i in range(len(p) - 1, -1, -1):
                while c < i and len(choices[p[c]]) == 1:
                    c += 1
                if c >= i:
                        break
                if len(choices[p[i]]) == 1:
                    p[c], p[i] = p[i], p[c]
                    c += 1
    #First round to attribute all the resident who have only one choice
    cont = True
    while cont:
        cont = False
        P1C = []
        for (p, r) in choices.items():
            if len(r) == 1:
                cont = True
                P1C.append((p, r[0]))
        if not cont:
            break
        random.shuffle(P1C)
        for (p, h) in P1C:
            if capacities[h] == 0:
                del_player(p, choices, hospital_preferences)
                continue
            capacities[h] -= 1
            assignments[h].append(p)
            del_player(p, choices, hospital_preferences)
        cleanup(choices, hospital_preferences, capacities)
    #Delete the hospital that are full, and restart while there are some residents with only one choice.

    #while there are still things waiting to be assigned
    cont = True
    while cont:
        cont = False
        #create a matching
        game = HospitalResident.create_from_dictionaries(choices, hospital_preferences, capacities)
        matching = game.solve(optimal="resident")
        #process it
        for (h, ass) in matching.items():
            for p in ass:
                remove_conflict(choices[str(p)], conflits, h)
            #for some reason the output of matching are not strings anymore
            h = str(h)
            #add it to the assignments
            assignments[h] += ass
            #substract the capacities
            capacities[h] -= len(ass)
            #prevent a player from getting assigned to the same hospital twice
            for p in ass:
                p = str(p)
                hospital_preferences[h].remove(p)
                choices[p].remove(h)
        #shuffle the hospital preferences and see if it should continue
        for (h, p) in hospital_preferences.items():
            random.shuffle(hospital_preferences[h])
            if capacities[h] > 0 and len(p) > 0:
                cont = True
        cleanup(choices, hospital_preferences, capacities)
    return assignments

In [56]:
choices = {"j1":["a1", "a2", "a3"], "j2":["a2","a3"], "j3":["a1"]}
cap = {"a1":1,"a2":1, "a3":10}
conflits = {"a2":["a3"], "a1":[], "a3":[]}
print(match(choices, cap, conflits))

{'a1': ['j3'], 'a2': [j2], 'a3': [j1]}


In [57]:
import csv
import pprint
def charger_activites():
    participants = dict()
    choix = dict()
    activites = dict()
    nb_demandes = dict()
    capacites = dict()
    conflits = dict()
    def supprimer_activite(id):
        for (p, c) in choix.items():
            if id in c:
                c.remove(id)
        del activites[id]
        del nb_demandes[id]
        if id in capacites:
            del capacites[id]

    with open('choix_activite_interludes', newline='') as csvfile:
        fichiers_choix = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in fichiers_choix:
            if row[0] == "id_participant":
                continue
            participants[row[0]] = row[1]
            if row[0] not in choix:
                choix[row[0]] = []
            if row[6] not in nb_demandes:
                nb_demandes[row[6]] = 0
            nb_demandes[row[6]] += 1
            choix[row[0]].insert(int(row[3]), row[6])
            activites[row[6]] = row[5]
    to_remove = []
    pprint.pp(activites, sort_dicts=True)
    for (k, n) in activites.items():
        max_participants = input("Nombre de places pour " + str(n))
        if nb_demandes[k] <= int(max_participants):
            print(n + " n'est pas complet, elle a été demandée " + str(nb_demandes[k]) + " fois")
            to_remove.append(k)
        else:
            capacites[k] = int(max_participants)
            conflits[k] = input("id des activites entrant en conflit avec " + str(n) + ", séparés par des espaces").split()
    for k in to_remove:
        supprimer_activite(k)

    return participants, choix, activites, conflits, capacites

In [58]:
participants, choix, activites, conflits, capacites = charger_activites()

{'1': 'CosmiA',
 '10': 'T. Geoguessr',
 '11': 'T. Pokémon Showdown',
 '2': 'Elémentaire !',
 '21': 'Chaîne Alimentaire',
 '23': 'Minecraft',
 '28': 'The space between us - Séance 1',
 '3': 'Documents Secrets',
 '4': 'The space between us - Séance 2',
 '5': 'Zero Escape : Radical Outcome',
 '6': 'Demi-Dieux - Séance 1',
 '7': 'Demi-Dieux - Séance 2',
 '8': 'Justice et Chevalerie',
 '9': 'Monster of the Week'}
Minecraft n'est pas complet, elle a été demandée 16 fois
Zero Escape : Radical Outcome n'est pas complet, elle a été demandée 7 fois
T. Geoguessr n'est pas complet, elle a été demandée 14 fois
T. Pokémon Showdown n'est pas complet, elle a été demandée 8 fois
CosmiA n'est pas complet, elle a été demandée 4 fois


In [60]:
capacites = dict()
for (a, n) in activites.items():
    capacites[a] = int(input("capacite pour " + n))

In [64]:
matchings = match(choix, capacites, conflits)

In [66]:
named_matchings = dict()
for (act_id, pl) in matchings.items():
    act = activites[str(act_id)]
    n_p = []
    for p in pl:
        n_p.append(participants[str(p)])
pprint.pp(named_matchings, sort_dicts=True)

{}
